In [12]:
!pip install jsons
!pip install jsonlib
!pip install tensorflow
!pip install sklearn

You should consider upgrading via the 'c:\users\elcot\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.

  ERROR: Command errored out with exit status 1:

  Using cached jsonlib-1.6.1.tar.gz (43 kB)

   command: 'c:\users\elcot\appdata\local\programs\python\python39\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ELCOT\\AppData\\Local\\Temp\\pip-install-xnmok7lf\\jsonlib_9bf669bc090249b997cddd749c96d497\\setup.py'"'"'; __file__='"'"'C:\\Users\\ELCOT\\AppData\\Local\\Temp\\pip-install-xnmok7lf\\jsonlib_9bf669bc090249b997cddd749c96d497\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\ELCOT\AppData\Local\Temp\pip-wheel-uh0bknbp'
       cwd: C:\Users\ELCOT\AppData\Local\Temp\pip-install-xnmok7lf\jsonlib_9bf669bc090249b997cddd749c96d497\
  Complete output (9 lines):
  running bdist_wheel

  Preparing metadata (setup.py): started

...

  creating build\lib.win-amd64-3.9


    Running setup.py install for jsonlib: finished with status 'error'

  copying jsonlib.py -> build\lib.win-amd64-3.9


  running build_ext
  building '_jsonlib' extension
  error: Microsoft Visual C++ 14.0 is required. Get it with "Build Tools for Visual Studio": https://visualstudio.microsoft.com/downloads/
  ----------------------------------------
  ERROR: Failed building wheel for jsonlib
    ERROR: Command errored out with exit status 1:
     command: 'c:\users\elcot\appdata\local\programs\python\python39\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ELCOT\\AppData\\Local\\Temp\\pip-install-xnmok7lf\\jsonlib_9bf669bc090249b997cddd749c96d497\\setup.py'"'"'; __file__='"'"'C:\\Users\\ELCOT\\AppData\\Local\\Temp\\pip-install-xnmok7lf\\jsonlib_9bf669bc090249b997cddd749c96d497\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --re

You should consider upgrading via the 'c:\users\elcot\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.

In [2]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [3]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [5]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [6]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
____________________________________________________

In [7]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 5s 188ms/step - loss: 2.0793 - accuracy: 0.0909
Epoch 2/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0777 - accuracy: 0.0909
Epoch 3/500
2/2 [==============================] - 0s 10ms/step - loss: 2.0774 - accuracy: 0.1515
Epoch 4/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0773 - accuracy: 0.0909
Epoch 5/500
2/2 [==============================] - 0s 9ms/step - loss: 2.0771 - accuracy: 0.0909
Epoch 6/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0774 - accuracy: 0.0909
Epoch 7/500
2/2 [==============================] - 0s 10ms/step - loss: 2.0776 - accuracy: 0.0909
Epoch 8/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0776 - accuracy: 0.0909
Epoch 9/500
2/2 [==============================] - 0s 10ms/step - loss: 2.0777 - accuracy: 0.0909
Epoch 10/500
2/2 [==============================] - 0s 7ms/step - loss: 2.0776 - accuracy: 0.0909
Epoch 11/500
2/2 [======

2/2 [==============================] - 0s 12ms/step - loss: 1.8601 - accuracy: 0.2121
Epoch 167/500
2/2 [==============================] - 0s 12ms/step - loss: 1.8571 - accuracy: 0.2424
Epoch 168/500
2/2 [==============================] - 0s 12ms/step - loss: 1.8549 - accuracy: 0.3030
Epoch 169/500
2/2 [==============================] - 0s 10ms/step - loss: 1.8529 - accuracy: 0.3030
Epoch 170/500
2/2 [==============================] - 0s 9ms/step - loss: 1.8507 - accuracy: 0.3030
Epoch 171/500
2/2 [==============================] - 0s 12ms/step - loss: 1.8480 - accuracy: 0.3030
Epoch 172/500
2/2 [==============================] - 0s 16ms/step - loss: 1.8453 - accuracy: 0.3030
Epoch 173/500
2/2 [==============================] - 0s 11ms/step - loss: 1.8431 - accuracy: 0.3333
Epoch 174/500
2/2 [==============================] - 0s 16ms/step - loss: 1.8410 - accuracy: 0.3333
Epoch 175/500
2/2 [==============================] - 0s 15ms/step - loss: 1.8389 - accuracy: 0.3333
Epoch 176/500
2

2/2 [==============================] - 0s 10ms/step - loss: 1.3312 - accuracy: 0.5152
Epoch 331/500
2/2 [==============================] - 0s 9ms/step - loss: 1.3284 - accuracy: 0.5152
Epoch 332/500
2/2 [==============================] - 0s 15ms/step - loss: 1.3253 - accuracy: 0.5152
Epoch 333/500
2/2 [==============================] - 0s 10ms/step - loss: 1.3227 - accuracy: 0.5152
Epoch 334/500
2/2 [==============================] - 0s 9ms/step - loss: 1.3198 - accuracy: 0.5152
Epoch 335/500
2/2 [==============================] - 0s 12ms/step - loss: 1.3170 - accuracy: 0.4848
Epoch 336/500
2/2 [==============================] - 0s 15ms/step - loss: 1.3145 - accuracy: 0.4242
Epoch 337/500
2/2 [==============================] - 0s 13ms/step - loss: 1.3115 - accuracy: 0.3939
Epoch 338/500
2/2 [==============================] - 0s 12ms/step - loss: 1.3088 - accuracy: 0.4848
Epoch 339/500
2/2 [==============================] - 0s 10ms/step - loss: 1.3063 - accuracy: 0.4848
Epoch 340/500
2/

2/2 [==============================] - 0s 29ms/step - loss: 0.8876 - accuracy: 0.7576
Epoch 495/500
2/2 [==============================] - 0s 11ms/step - loss: 0.8868 - accuracy: 0.7273
Epoch 496/500
2/2 [==============================] - 0s 11ms/step - loss: 0.8887 - accuracy: 0.6970
Epoch 497/500
2/2 [==============================] - 0s 12ms/step - loss: 0.8907 - accuracy: 0.6970
Epoch 498/500
2/2 [==============================] - 0s 18ms/step - loss: 0.8950 - accuracy: 0.6970
Epoch 499/500
2/2 [==============================] - 0s 15ms/step - loss: 0.8993 - accuracy: 0.6970
Epoch 500/500
2/2 [==============================] - 0s 11ms/step - loss: 0.9007 - accuracy: 0.6667


In [8]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


In [9]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hi
ChatBot: Hi there
User: how are you
ChatBot: I'm Joana, an Artificial Intelligent bot
User: nice to meet you
ChatBot: Hello
User: bye
ChatBot: Hi
User: quit
